In [7]:
import pandas as pd
import numpy as np
import matplotlib#.pyplot as plt
import flask

In [8]:
print(matplotlib.__version__)

3.4.2


In [2]:
messages=pd.read_csv("untitled.txt")

In [3]:
messages=messages[["class","message"]]

In [4]:
messages.columns=["label","message"]

In [5]:
messages

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will �_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


# Data cleaning and preprocessing

In [2]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [7]:
ps=PorterStemmer()

In [8]:
corpus=[]
for i in range(len(messages)):
    review=re.sub("[^a-zA-Z]"," ",messages["message"][i])
    review=review.lower()
    review=review.split()
    review=[ps.stem(i) for i in review if not i in stopwords.words('english')]
    review=" ".join(review)
    corpus.append(review)

#  Creating the Bag of Words model

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=2500)
X=cv.fit_transform(corpus).toarray()
y=messages.label.map({"ham":1,"spam":0})

In [10]:
X[0].shape

(2500,)

(2500,)

In [11]:
import pickle
pickle.dump(cv,open("cv.pkl","wb"))

# Spliting the data

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# navie basie

In [13]:
from sklearn.naive_bayes import MultinomialNB
spam_detect_model = MultinomialNB().fit(X_train, y_train)

y_pred_naive_bayes=spam_detect_model.predict(X_test)

In [14]:
pickle.dump(spam_detect_model,open("spam_detect_model.pkl","wb"))

In [15]:
from sklearn.metrics import classification_report

In [16]:
print(classification_report(y_test,y_pred_naive_bayes))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       166
           1       0.99      0.99      0.99       949

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       166
           1       0.99      0.99      0.99       949

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115



# knn

In [18]:
#error_rate = []

# Will take some time
#for i in range(1,40):
#    
#    knn = KNeighborsClassifier(n_neighbors=i)
#    knn.fit(X_train,y_train)
#    pred_i = knn.predict(X_test)
#    error_rate.append(np.mean(pred_i != y_test))
#    
#plt.figure(figsize=(10,6))
#plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
#         markerfacecolor='red', markersize=10)
#plt.title('Error Rate vs. K Value')
#plt.xlabel('K')
#plt.ylabel('Error Rate')

# knn


In [19]:
#knn = KNeighborsClassifier(n_neighbors=1)
#knn.fit(X_train,y_train)
#y_pred_knn = knn.predict(X_test)
#knn.score(X_test,y_test)

In [20]:
print(classification_report(y_test,y_pred_naive_bayes))
print("naive")
print("----------------------------------------------------------------------------------")
#print(classification_report(y_test,y_pred_knn))
#print("knn")
#print("----------------------------------------------------------------------------------")

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       166
           1       0.99      0.99      0.99       949

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115

naive
----------------------------------------------------------------------------------
              precision    recall  f1-score   support

           0       0.96      0.95      0.95       166
           1       0.99      0.99      0.99       949

    accuracy                           0.99      1115
   macro avg       0.98      0.97      0.97      1115
weighted avg       0.99      0.99      0.99      1115

naive
----------------------------------------------------------------------------------


# test app.py

In [21]:
import pickle

In [22]:
cv=pickle.load(open("cv.pkl","rb"))
spam_detect_model=pickle.load(open("spam_detect_model.pkl","rb"))

In [40]:
data = ["hello bro"]
vect = cv.transform(data).toarray()
my_prediction = spam_detect_model.predict(vect)[0]
my_prediction

1

1

# app.py

In [42]:
from flask import Flask,render_template,request
import pickle

app=Flask(__name__)

@app.route("/")
def home():
    return render_template("index.html")

@app.route("/predict",methods=["POST"])
def predict():
    message=request.form['message']
    data=[message]
    vect = cv.transform(["this is spam"]).toarray()
    my_prediction = spam_detect_model.predict(vect)[0]
    #{"ham":1,"spam":0}
    print(my_prediction)
    if int(my_prediction)==1:
        my_prediction="it's a ham"
    else:
        my_prediction="it's a spam"
        
    return render_template("index.html",output=my_prediction)
if __name__=="__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
